In [1]:
import requests
from selenium import webdriver
from bs4 import BeautifulSoup as BS
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
import pandas as pd

In [2]:
BASE_URL = "https://www.imdb.com/search/title/?title_type=feature"

In [3]:
webdriver.ChromeOptions().add_argument('--headless')
driver = webdriver.Chrome()
driver.maximize_window()
driver.get(BASE_URL)

In [6]:
page_source = driver.page_source
soup = BS(page_source, "html.parser")

In [7]:
import time
for _ in range(30):
    btn = driver.find_elements(By.XPATH, '//*[@id="__next"]/main/div[2]/div[3]/section/section/div/section/section/div[2]/div/section/div[2]/div[2]/div[2]/div[1]/span/button')
    btn = btn[0]
    driver.execute_script("arguments[0].scrollIntoView();", btn)
    try:
        btn.click()
    except Exception:
        pass
    time.sleep(1)
print('Done')

In [8]:
soup = BS(driver.page_source.encode('utf-8'), "html.parser")

In [9]:
movies_list_ul = soup.find("ul", {"class": "ipc-metadata-list"})

In [10]:
movies_list_with_title = []
for i in movies_list_ul:
    title = i.h3.text
    title = (title.split('.')[1]).strip()
    movies_list_with_title.append({'title': title, 'link': i.a['href']})
    

In [11]:
df = pd.DataFrame(movies_list_with_title)

In [12]:
df.to_csv('movies_list_with_title.csv', sep=",", index=False)

In [13]:
df = pd.read_csv('movies_list_with_title.csv')

In [14]:
class ScrapingHelper:
    """
    A helper class which helps the main scraping function to be maintainable and readable.
    """
    
    @staticmethod
    def get_details(soup):
        """
        function from scraping released year, rated class and movie run time
        """
        released_year, rated, run_time = (None, None, None)
        top_ul =  soup.find('ul', {'class':'cdJsTz'})
        if top_ul:
            lis = top_ul.find_all('li')
            for i in lis:
                if i.a:
                    if 'parentalguide' in i.a['href']:
                        rated = i.a.text
                    if 'releaseinfo' in i.a['href']:
                        released_year = i.a.text
                else:
                    run_time = i.text
        return released_year, rated, run_time
        
    
    @staticmethod
    def find_stars_and_total_ratings(soup):
        stars = soup.find('span', {'class':'cMEQkK'})
        if stars:
            stars = stars.text
        else:
            stars = None

        total_ratings = soup.find('div', {'class':'gPVQxL'})
        total_ratings = total_ratings.text if total_ratings else None
        
        return stars, total_ratings
    
    @staticmethod
    def get_genre(soup):
        genres_list = []
        genres = soup.find_all('span', class_='ipc-chip__text')
        for i in genres:
            if i.text != 'Back to top':
                genres_list.append(i.text)
        return "|".join(genres_list)
    
    @staticmethod
    def get_summary(soup):
        summary =  soup.find('span', {'class':'chnFO'})
        return summary.text if summary else None
    
    

In [15]:
# movies map to store data for our dataframe
movies = {
    'title': [],
    'released_year': [],
    'rated_class': [],
    'run_time': [],
    'stars': [],
    'total_ratings': [],
    'genre': [],
    'summary': []
}

In [16]:
BASE_URL = "https://imdb.com"
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
for _, m in df[3079:].iterrows():
    full_url = f"{BASE_URL}{m['link']}"
    data = requests.get(full_url, headers=headers)
    soup = BS(data.text, 'html.parser')
    released_year, rated, run_time = ScrapingHelper.get_details(soup)
    stars, total_ratings = ScrapingHelper.find_stars_and_total_ratings(soup)
    genre = ScrapingHelper.get_genre(soup)
    summary = ScrapingHelper.get_summary(soup)
    
    # appending to our map
    movies['title'].append(m['title'])
    movies['released_year'].append(released_year)
    movies['rated_class'].append(rated)
    movies['run_time'].append(run_time)
    movies['stars'].append(stars)
    movies['total_ratings'].append(total_ratings)
    movies['genre'].append(genre)
    movies['summary'].append(summary)

In [17]:
movies = pd.DataFrame(movies)
movies.to_csv('final_movies.csv', sep=",")